## Face detection

La ProCam s.p.a ha intenzione di lanciare sul mercato una nuova fotocamera digitale compatta ed economica destinata a piccoli fotografi in erba.

Vieni assunto come Data Scientist per realizzare il sistema di identificazione dei volti nelle immagini, questo permetterà poi ai tecnici della fotografia di ottimizzare le impostazioni per un selfie con una o più persone.

Si tratta di un problema di computer vision, più precisamente di Face Detection.

Devi fornire una pipeline scikit-learn che prende un'immagine in ingresso e ritorna una lista con le coordinate dei bounding box dove sono presenti dei volti, se nell'immagine non contiene volti la lista sarà ovviamente vuota.

- Non ti viene fornito un dataset, sta a te cercarne uno in rete o, nella peggiore delle ipotesi, costruirlo, per semplicità non considereremo implicazioni sulle licenze ad utilizzo commerciale, si tratta pur sempre di un progetto didattico.
- Non puoi utilizzare modelli pre-addestrati, devi addestrarlo tu utilizzando scikit-learn.
- Stai lavorando su un sistema con ridotte capacità di calcolo, quindi il modello deve richiedere poche risorse di calcolo.
- Ovviamente non ti vengono fornite indicazioni sull'implementazione, fai un'approfondita ricerca bibliografica per trovare la soluzione migliore da adottare, il notebook che consegnerai deve essere ben documentato, devi spiegare quali soluzioni hai adottato e perché ed ogni risorsa esterna (paper, blog post, codice github...) che hai utilizzato.
- Il progetto è abbastanza complesso, ricorda che in caso ne avessi necessità puoi sempre chiedere aiuto ai tuoi coach nella Classe Virtuale di Machine Learning su Discord.

https://www.analyticsvidhya.com/blog/2019/09/feature-engineering-images-introduction-hog-feature-descriptor/

https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html

Import object images from kaggle

In [ ]:
# !pip install kaggle

Butterfly dataset (832 images). 

It contains bufferflies images with some flowers and plants. With this dataset model can learn to detect naturalistic objects.

In [ ]:
!kaggle datasets download veeralakrishna/butterfly-dataset

In [ ]:
# !unzip butterfly-dataset.zip -d butterfly-dataset
# !tar -xf butterfly-dataset.zip

Background dataset (715 images).

It contains some backgournd images, taken from streets and landscape photos. With this dataset model can learn to detect common background objects and patterns.

In [ ]:
!mkdir stanford-background-dataset

In [ ]:
!kaggle datasets download -p stanford-background-dataset balraj98/stanford-background-dataset 

In [ ]:
# !unzip stanford-background-dataset.zip -d stanford-background-dataset
# !tar -xf stanford-background-dataset/stanford-background-dataset.zip -C stanford-background-dataset

Add some animals pcitures :)

In [ ]:
!mkdir animals

In [ ]:
!kaggle datasets download -p animals alessiocorrado99/animals10

In [ ]:
# !tar -xf animals/animals10.zip -C animals

In [ ]:
#Import some modules
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage.transform import resize
from skimage.io import imread
from skimage.exposure import rescale_intensity
from sklearn.datasets import fetch_lfw_people
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from ImageResizer import ImageResizer
from HOGFeatureExtractor import HOGFeatureExtractor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import lognorm
from scipy.stats import genhalflogistic
from scipy.stats import powerlaw


In [ ]:
RANDOM_SEED=200

Prima di tutto provo a leggere le immagini di esempio di sklearn e provo ad estrarre hog features da questo... Il resto dopo

In [ ]:
#impoort first dataset (faces)
lfw_people = fetch_lfw_people(resize=1)

images_positive = lfw_people.images
#use only a portion of these images, to don't unbalance final dataset
# indexes = np.random.choice(len(lfw_people.images),1600, replace=False)
# images_positive = lfw_people.images[indexes]

#some helper variables
#size = 64 x 128 as original paper
resize_shape = (128, 64) #as row x columns (h x w)

#init list to hold features arrays and labels
features_list = []
labels_list = []

In [ ]:
X_positive = images_positive
y_positive = np.ones(X_positive.shape[0])
print(X_positive.shape) #3 dimensions: 1 = records, 2-3 = image as matrix 
print(y_positive.shape)

In [ ]:
print(X_positive[1,:,:].shape)

In [ ]:
#process negative images
negative_images = []

butterflies_img_dir = 'leedsbutterfly/images'
background_img_dir = 'stanford-background-dataset/images'
cat_img_dir = 'animals/raw-img/gatto'
chicken_img_dir = 'animals/raw-img/gallina'
cow_img_dir = 'animals/raw-img/mucca'
squirrel_img_dir = 'animals/raw-img/scoiattolo'
sheep_img_dir = 'animals/raw-img/pecora'
negative_img_dirs = [butterflies_img_dir,
                     background_img_dir,
                     cat_img_dir,
                     chicken_img_dir,
                     cow_img_dir,
                     squirrel_img_dir,
                     sheep_img_dir                    
                    ]

# for directory in tqdm(negative_img_dirs,desc="Dataset:",unit="item"):
for directory in negative_img_dirs:
    for filename in tqdm(os.listdir(directory),desc=f"Processing negative images ({directory})",
                        unit="item"):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
            img_path = os.path.join(directory, filename)
            img = imread(img_path, as_gray=True)
            #this resize is necessary for concatenation in a single numpy array
            img = resize(img, X_positive[1,:,:].shape)
            negative_images.append(img)

X_negative = np.array(negative_images)
y_negative = np.zeros(X_negative.shape[0])
print(X_negative.shape) #3 dimensions: 1 = records, 2-3 = image as matrix 
print(y_negative.shape)

In [ ]:
X = np.vstack((X_positive, X_negative))
y = np.concatenate([y_positive,y_negative])

print(X.shape)
print(y.shape)

Primo tentativo di pipeline...

In [ ]:
resize_shape = (128, 64) #as row x columns (h x w)

# Definisci la pipeline
pipeline = Pipeline(steps=[
    ('resizer', ImageResizer(resize_shape)),
    ('hog', HOGFeatureExtractor()),
    ('scaler', StandardScaler()),
    # ('pca', PCA(n_components=100)),  # Scegli il numero di componenti che desideri
    ('pca', PCA(svd_solver="full")), 
    ('svc', SVC())  # SVC con kernel polinomiale
], memory="pipe_cache")


# Addestra il modello (edit: si fa con il random search)
#pipeline.fit(X, y)

pipeline

Random search optimization

Effetto di C (deve essere positivo):
- se C aumenta, ho più classificazioni corrette, ho margine più stretto e quindi meno vettori di supporto (fit più lungo per cercare i SV)
- se C diminuiusce, ho più errori, ho margine più largo e quindi ho più vettori di supporto (predizione più lunga)

Effetto di gamma (deve essere positivo):
- se gamma aumenta, i vettori di supporto influenzano una zona più ristretta, quindi ottengo un decision boundary più accartocciato attorno ai SV (quindi più probabile overfitting)
- se gamma diminuisce i SV influenzano una zona più larga, ho un decision boundary più liscio e semplice (perdo la forma dei dati, underfitting)

https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html#sphx-glr-auto-examples-svm-plot-rbf-parameters-py

In [ ]:
#uniform distribution, such that all values are equally probable
C_range = uniform(loc=0.001, scale=100.0)
#lognorm distribution, such that lower values are more probable (if gamma is too high, decision boundary is too close to SV and i have overfitting)
gamma_range = lognorm(s=0.95, loc=0, scale=1)
degree_range = list(range(3,11))
#i want negative skewness here (higher values are preferred)
n_components_range = powerlaw(3.7)

grid = [{
    "pca__n_components":n_components_range,
    "svc__kernel" : ["rbf"],
    "svc__gamma" : gamma_range,
    "svc__C" : C_range
}]
# {
#     "pca__n_components":n_components_range,
#     "svc__kernel" : ["poly"],
#     "svc__gamma" : gamma_range,
#     "svc__C" : C_range,
#     "svc__degree": degree_range
# }]

search = RandomizedSearchCV(estimator=pipeline,
                            param_distributions=grid,
                            n_iter=20,
                            cv=10,
                            scoring="accuracy",
                            verbose=4,
                            random_state=RANDOM_SEED,
                            n_jobs=1)

# x = np.linspace(genhalflogistic.ppf(0.01, 1),
#                 genhalflogistic.ppf(0.99, 1), 100)
# plt.plot(x, genhalflogistic.pdf(x, 1),
#        'r-', lw=5, alpha=0.6, label='genhalflogistic pdf')

# plt.show()

# b=3.7
# x = np.linspace(powerlaw.ppf(0.01, b),
#                 powerlaw.ppf(0.99, b), 100)
# plt.plot(x, powerlaw.pdf(x, b),
#        'r-', lw=5, alpha=0.6, label='powerlaw pdf')

# plt.show()



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=RANDOM_SEED)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# pipeline.fit(X_train, y_train)
search.fit(X_train, y_train)

In [ ]:
print(f"Best parameters: {search.best_params_}")
print(f"Best accuracy: {search.best_score_}")

In [ ]:
search.best_estimator_

In [ ]:

# print(pipeline.predict([X[-100,:,:]]))

# TODO: Inserire il random searchCV con la pipeline custom creata sopra

https://scikit-learn.org/stable/modules/compose.html

https://scikit-learn.org/stable/auto_examples/compose/plot_digits_pipe.html#sphx-glr-auto-examples-compose-plot-digits-pipe-py

https://scikit-learn.org/stable/auto_examples/compose/plot_compare_reduction.html#sphx-glr-auto-examples-compose-plot-compare-reduction-py